In [1]:
# Importieren der Bibliotheken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import contextlib
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error

In [2]:
# Datensätze laden
daten1950=pd.read_csv("daten1950.csv").drop(["Unnamed: 0"], axis=1)[:-1]
daten1973=pd.read_csv("daten1973.csv").drop(["Unnamed: 0"], axis=1)[:-1]
daten1990=pd.read_csv("daten1990.csv").drop(["Unnamed: 0"], axis=1)[:-1]
daten2006=pd.read_csv("daten2006.csv").drop(["Unnamed: 0"], axis=1)[:-1]

In [71]:
# Funktion der Prognosemodelle
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird. 
# methode: "lr", "pr", "tree", "nn", "lstm", "sv", "knn", "arima", "arimax" - Das zu nutzende Modell
# ensemble: "bagging", "ada_boosting" - Nutzung der Ensemble-Algorithmen
# x/y test/train: Trainings- und Testdaten. Werden über die Funktion compare_result() automatisch übergeben
def modell(methode, ensemble, x_train, x_test, y_train, y_test, daten):
    y_pred = None  # Vorab-Initialisierung von y_pred
#     ensemble = None

    # Lineare Regression
    if methode == "lr":
        model = LinearRegression()

    # Polynom Regression
    elif methode == "pr":
        # Polynomiale Merkmale erstellen
        if daten.equals(daten1950):
            poly_features = PolynomialFeatures(degree=2)
        elif daten.equals(daten1973):
            poly_features = PolynomialFeatures(degree=4)
        elif daten.equals(daten1990):
            poly_features = PolynomialFeatures(degree=2)
        elif daten.equals(daten2006):
            poly_features = PolynomialFeatures(degree=2)
        else: 
            pass
        
        x_train = poly_features.fit_transform(x_train)
        x_test = poly_features.transform(x_test)

        # Modell erstellen und trainieren
        model = LinearRegression()
        
    # Entscheidungsbaum
    elif methode == "tree":
        model = DecisionTreeRegressor(random_state=40)

 # Neuronales Netzwerk
    elif methode == "nn":
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(124, activation='relu', input_shape=(x_train.shape[1],)),
            tf.keras.layers.Dense(124, activation='relu'),
            tf.keras.layers.Dense(124, activation='relu'),
            tf.keras.layers.Dense(62, activation='relu'),
            tf.keras.layers.Dense(1)  # Ausgabeschicht
        ])

        # Modell kompilieren
        model.compile(loss='mean_absolute_error', optimizer='adam')
        
        # Modell trainieren
        model.fit(x_train, y_train, epochs=200, batch_size=32, verbose=False)

        y_pred = model.predict(x_test).flatten()

    # LSTM
    elif methode == "lstm":
        # Umwandeln der Daten in das erforderliche 3D-Format für LSTM
        x_train_lstm = np.reshape(x_train.values, (x_train.shape[0], 1, x_train.shape[1]))
        x_test_lstm = np.reshape(x_test.values, (x_test.shape[0], 1, x_test.shape[1]))

        # Modell erstellen
        model = Sequential()
        model.add(LSTM(124, activation='relu', return_sequences=True, input_shape=(1, x_train.shape[1])))
        model.add(LSTM(124, activation='relu', return_sequences=True))
        model.add(LSTM(124, activation='relu', return_sequences=True))
        model.add(LSTM(62, activation='relu', return_sequences=True))
        model.add(Dense(1))

        # Modell kompilieren
        model.compile(loss='mean_squared_error', optimizer='adam')
            
        # Modell trainieren
        model.fit(x_train_lstm, y_train, epochs=200, batch_size=32, verbose=False)

        # Anpassen für die Weitere Analyse
        x_test = x_test_lstm
        x_train = x_train_lstm

        y_pred = model.predict(x_test).flatten()


    # Support Vector Regression
    elif methode == "sv":
        # Modellparameter festlegen
        if daten.equals(daten1950):
            model = SVR(kernel='linear', C=300, epsilon=1)
        elif daten.equals(daten1973):
            model = SVR(kernel='linear', C=300, epsilon=0.1)
        elif daten.equals(daten1990):
            model = SVR(kernel='linear', C=300, epsilon=1)
        elif daten.equals(daten2006):
            model = SVR(kernel='linear', C=300, epsilon=1)
            
    # KNN-Regression
    elif methode == "knn":
        # Modell initialisieren und trainieren
        model = KNeighborsRegressor(n_neighbors=1, p=1, weights="distance")

    # Arima
    elif methode == "arima":
        # ARIMA-Modell erstellen und anpassen
        if daten.equals(daten1950):
            model = ARIMA(y_train, order=(3, 1, 2))
        elif daten.equals(daten1973):
            model = ARIMA(y_train, order=(7, 1, 1))
        elif daten.equals(daten1990):
            model = ARIMA(y_train, order=(2, 1, 1))
        elif daten.equals(daten2006):
            model = ARIMA(y_train, order=(2, 1, 5))
        else:
            pass
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=4)

    # Arimax
    elif methode == "arimax":
        # ARIMAX-Modell erstellen und anpassen
        if daten.equals(daten1950):
            model = ARIMA(y_train, order=(10, 4, 1), exog=x_train)
        elif daten.equals(daten1973):
            model = ARIMA(y_train, order=(3, 3, 1), exog=x_train)
        elif daten.equals(daten1990):
            model = ARIMA(y_train, order=(1, 1, 2), exog=x_train)
        elif daten.equals(daten2006):
            model = ARIMA(y_train, order=(2, 1, 1), exog=x_train)
        else:
            pass
        
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=4, exog=x_test)

    else:
        pass


    # Bagging einfügen
    if ensemble == "bagging":
        model = BaggingRegressor(model, random_state=42)
        # Modell trainieren
        model.fit(x_train, y_train)
        # Vorhersagen für Testdaten
        y_pred = model.predict(x_test).flatten()

    # ADA Boosting einfügen
    elif ensemble == "ada_boosting":
        boosting_model = AdaBoostRegressor(base_estimator=model, random_state=42)
        # Modell trainieren
        boosting_model.fit(x_train, y_train)
        # Vorhersagen für Testdaten
        y_pred = boosting_model.predict(x_test).flatten()

    # Modellprognosen ohne Bagging, Boosting oder ADA Boosting
    elif ensemble != "bagging" and ensemble != "ada_boosting" and methode != "nn" and methode != "lstm" and methode != "arima" and methode != "arimax":
        # Modell trainieren
        model.fit(x_train, y_train)
        # Vorhersagen für Testdaten
        y_pred = model.predict(x_test).flatten()

    else:
        pass

    return y_pred

    
# Komponentenanzahl für PCA finden & speichern
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird. 
# pca_ausgaben: 1 ist die Ausgabe der Schleife mit verschiedener anzahl an Hauptkomponenten, 2 ist die Rückgabe der PCA Tabellen zur weiterverarbeitung
# methode: "lr", "pr", "tree", "nn", "lstm", "sv", "knn", "arima", "arimax" - Das zu nutzende Modell
# ensemble: "bagging", "ada_boosting" - Nutzung der Ensemble-Algorithmen
# split: "validierung", "test" - Definiert, ob der Validierungs- oder Testzeitraum mit den zugehörigen Daten genutzt werden
# normalize: True, False - Ob Daten normalisiert werden
def find_best_pca(daten, pca_ausgaben, methode, ensemble, split, normalize=True):
    # PCA - Dimensionen der Datensätze reduzieren
    x = daten.drop(["co2"], axis=1)

    if normalize:
        x = StandardScaler().fit_transform(x)
        x = pd.DataFrame(x, columns=daten.columns.drop("co2"))

    # Liste für MAE-Werte
    mae_values = []
    
    if split == "validierung": 
        zeit1=-8
        zeit2=-4
    elif split == "test":
        zeit1=-4
        zeit2=None
    else:
        pass

    # Iteration über verschiedene Anzahlen von PCA-Komponenten
    for n in range(2, 11):
        # PCA durchführen
        pca = PCA(n_components=n)
        PC = pca.fit_transform(x)
        principalDF = pd.DataFrame(data=PC, columns=[f'pc{i}' for i in range(1, n + 1)])
        pca_df = pd.concat([principalDF, daten[['co2']]], axis=1)
        
        # Festlegen des Trainings- und Testdatenbereichs
        x_train = pca_df.drop(["co2"], axis=1)[:zeit1]
        x_test = pca_df.drop(["co2"], axis=1)[zeit1:zeit2]
        y_train = pca_df['co2'][:zeit1]
        y_test = pca_df['co2'][zeit1:zeit2]
        

        # Ausgewählte Modellprognose
        y_pred = modell(methode, ensemble, x_train, x_test, y_train, y_test, daten)

        # Berechnung des Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)
        mae_values.append(mae)

    # Ausgabe der MAE-Werte
    for n, mae in zip(range(2, 11), mae_values):
        print(f"Anzahl der PCA-Komponenten: {n}, MAE: {mae}")

    # Index des niedrigsten MAE-Werts ermitteln
    min_mae_index = np.argmin(mae_values)

    # PCA mit niedrigstem MAE
    best_pca = PCA(n_components=min_mae_index + 2)
    PC = best_pca.fit_transform(x)
    principalDF = pd.DataFrame(data=PC, columns=[f'pc{i}' for i in range(1, min_mae_index + 3)])
    pca_df = pd.concat([principalDF, daten[['co2']]], axis=1)

    # Gibt mit pca_ausgaben==1 die MAE über alle Iterationen wieder und mit pca_ausgaben==2 speichert es die beste PCA für die weitere Analyse
    if pca_ausgaben==1:
        return print("Niedrigster MAE:", min_mae_index+2)
    elif pca_ausgaben==2:
        return best_pca, pca_df
    else:
        pass

# Vergleiche die beste MAE mit den nicht-transformierten PCA Werten / Gebe die Statistiken und Ergebnisse der Prognosen durch
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird. 
# methode: "lr", "pr", "tree", "nn", "lstm", "sv", "knn", "arima", "arimax" - Das zu nutzende Modell
# ensemble: "bagging", "ada_boosting" - Nutzung der Ensemble-Algorithmen
# normalize: True, False - Ob Daten normalisiert werden
# split: "validierung", "test" - Definiert, ob der Validierungs- oder Testzeitraum mit den zugehörigen Daten genutzt werden
def compare_result(daten, methode, ensemble, split, normalize=True):
    # PCA mit niedrigstem MSE finden
    best_pca, pca_df = find_best_pca(daten, 2, methode, ensemble, split, normalize=normalize)

    if split == "validierung": 
        zeit1=-8
        zeit2=-4
    elif split == "test":
        zeit1=-4
        zeit2=None
    else:
        pass
    
    # Prognose mit PCA
    x_train_pca = pca_df.drop(["co2"], axis=1)[:zeit1]
    x_test_pca = pca_df.drop(["co2"], axis=1)[zeit1:zeit2]
    y_train_pca = pca_df['co2'][:zeit1]
    y_test_pca = pca_df['co2'][zeit1:zeit2]

    # Modell mit PCA trainieren
    y_pred_pca = modell(methode, ensemble, x_train_pca, x_test_pca, y_train_pca, y_test_pca, daten)
    
    # Prognose ohne PCA
    x_train = daten.drop(["co2"], axis=1)[:zeit1]
    x_test = daten.drop(["co2"], axis=1)[zeit1:zeit2]
    y_train = daten['co2'][:zeit1]
    y_test = daten['co2'][zeit1:zeit2]

    # Modell mit PCA trainieren
    y_pred = modell(methode, ensemble, x_train, x_test, y_train, y_test, daten)

    # MAE berechnen
    mae_pca = mean_absolute_error(y_test_pca, y_pred_pca)
    mae = mean_absolute_error(y_test, y_pred)

    # Ausgabe der MAE
    print("MAE mit PCA:", mae_pca)
    print("MAE mit PCA:", mae)

    # Neue Daten für einen Vergleich der Daten vorbereiten
    neue_tabelle = pd.DataFrame({'PCA y': y_pred_pca, "Co2 Emissionen": y_test_pca})
    d = {'jahr': daten["jahr"][zeit1:zeit2], 'Co2 Wert': daten["co2"][zeit1:zeit2], "PCA y": daten["co2"][zeit1:zeit2]}
    df = pd.DataFrame(data=d)
    df["PCA y"][-4:] = neue_tabelle["PCA y"]

    return df

In [43]:
# Hyperparameter sind für die Prognosen fixiert worden
# Nur die wichtigen Modelle wurden integriert
# PCA wird immer durchgeführt


# Funktion der Prognosemodelle
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird
# methode: "lr", "pr", "sv", "knn", "arima", "arimax" - Das zu nutzende Modell
# x/y test/train: Trainings- und Testdaten. Werden über die Funktion compare_result2() automatisch übergeben
def modell2(methode, x_train, x_test, y_train, y_test, daten):
    # Vorab-Initialisierungen
    y_pred = None
    ensemble = None

    # Lineare Regression
    if methode == "lr":
        model = LinearRegression()
        if daten.equals(daten1950):
            ensemble = "ada_boosting"
        elif daten.equals(daten1973):
            poly_features = PolynomialFeatures(degree=2)
        elif daten.equals(daten1990):
            poly_features = PolynomialFeatures(degree=2)
            ensemble = "bagging"
        elif daten.equals(daten2006):
            ensemble = "bagging"
        else: 
            pass

    # Polynom Regression
    elif methode == "pr":
        # Polynomiale Merkmale erstellen
        if daten.equals(daten1950):
            poly_features = PolynomialFeatures(degree=4)
        elif daten.equals(daten1973):
            poly_features = PolynomialFeatures(degree=2)
            ensemble = "bagging"
        elif daten.equals(daten1990):
            poly_features = PolynomialFeatures(degree=2)
            ensemble = "ada_boosting"
        elif daten.equals(daten2006):
            poly_features = PolynomialFeatures(degree=2)
        else: 
            pass
        
        x_train = poly_features.fit_transform(x_train)
        x_test = poly_features.transform(x_test)

        # Modell erstellen und trainieren
        model = LinearRegression()
        
    # Support Vector Regression
    elif methode == "sv":
        # Modellparameter festlegen
        if daten.equals(daten1950):
            model = SVR(kernel='linear', C=300, epsilon=1)
            ensemble = "ada_boosting"
        elif daten.equals(daten1973):
            model = SVR(kernel='linear', C=300, epsilon=0.1)
            ensemble = "ada_boosting"
        elif daten.equals(daten1990):
            model = SVR(kernel='linear', C=300, epsilon=1)
            ensemble = "bagging"
        elif daten.equals(daten2006):
            model = SVR(kernel='linear', C=300, epsilon=1)
            ensemble = "ada_boosting"
            
    # KNN-Regression
    elif methode == "knn":
        # Modell initialisieren und trainieren
        model = KNeighborsRegressor(n_neighbors=1, p=1, weights="distance")

    # Arima
    elif methode == "arima":
        # ARIMA-Modell erstellen und anpassen
        if daten.equals(daten1950):
            model = ARIMA(y_train, order=(3, 1, 2))
        elif daten.equals(daten1973):
            model = ARIMA(y_train, order=(7, 1, 1))
        elif daten.equals(daten1990):
            model = ARIMA(y_train, order=(2, 1, 1))
        elif daten.equals(daten2006):
            model = ARIMA(y_train, order=(2, 1, 5))
        else:
            pass
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=4)

    # Arimax
    elif methode == "arimax":
        # ARIMAX-Modell erstellen und anpassen
        if daten.equals(daten1950):
            model = ARIMA(y_train, order=(10, 4, 1), exog=x_train)
        elif daten.equals(daten1973):
            model = ARIMA(y_train, order=(3, 3, 1), exog=x_train)
        elif daten.equals(daten1990):
            model = ARIMA(y_train, order=(1, 1, 2), exog=x_train)
        elif daten.equals(daten2006):
            model = ARIMA(y_train, order=(2, 1, 1), exog=x_train)
        else:
            pass
        
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=4, exog=x_test)

    else:
        pass

    # Bagging einfügen
    if ensemble == "bagging":
        model = BaggingRegressor(model, random_state=42)
        # Modell trainieren
        model.fit(x_train, y_train)
        # Vorhersagen für Testdaten
        y_pred = model.predict(x_test).flatten()

    # ADA Boosting einfügen
    elif ensemble == "ada_boosting":
        boosting_model = AdaBoostRegressor(base_estimator=model, random_state=42)
        # Modell trainieren
        boosting_model.fit(x_train, y_train)
        # Vorhersagen für Testdaten
        y_pred = boosting_model.predict(x_test).flatten()

    # Modellprognosen ohne Bagging, Boosting oder ADA Boosting
    elif ensemble != "bagging" and ensemble != "boosting" and ensemble != "ada_boosting" and methode != "nn" and methode != "lstm" and methode != "arima" and methode != "arimax":
        # Modell trainieren
        model.fit(x_train, y_train)
        # Vorhersagen für Testdaten
        y_pred = model.predict(x_test).flatten()

    else:
        pass

    return y_pred

# Anzahl der Hauptkomponenten fixieren
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird
# methode: "lr", "pr", "sv", "knn", "arima", "arimax" - Das zu nutzende Modell
# normalize: True, False - Ob Daten normalisiert werden
def find_best_pca(daten, methode, normalize=True):
    # PCA - Dimensionen der Datensätze reduzieren
    x = daten.drop(["co2"], axis=1)

    if normalize:
        x = StandardScaler().fit_transform(x)
        x = pd.DataFrame(x, columns=daten.columns.drop("co2"))
    
    if daten.equals(daten1950):
        if methode in ["lr", "sv"]:
            n_components = 7
        elif methode in ["pr", "knn", "arima"]:
            n_components = 2
        elif methode in ["arimax"]:
            n_components = 3
        else:
            pass
    
    elif daten.equals(daten1973):
        if methode in ["lr", "arimax"]:
            n_components = 7
        elif methode in ["knn", "arima"]:
            n_components = 2
        elif methode in ["pr"]:
            n_components = 5
        elif methode in ["sv"]:
            n_components = 6
        else:
            pass
        
    elif daten.equals(daten1990):
        if methode in ["lr", "arimax"]:
            n_components = 5
        elif methode in ["pr", "knn"]:
            n_components = 2
        elif methode in ["sv", "arima"]:
            n_components = 6
        else:
            pass
    
    elif daten.equals(daten2006):
        if methode in ["lr", "sv", "knn"]:
            n_components = 2
        elif methode in ["pr"]:
            n_components = 8
        elif methode in ["arimax", "arima"]:
            n_components = 6
        else:
            pass
    else:
        pass
    
    # PCA anzahl bestimmen
    best_pca = PCA(n_components)
    PC = best_pca.fit_transform(x)
    principalDF = pd.DataFrame(data=PC)
    pca_df = pd.concat([principalDF, daten[['co2']]], axis=1)
    
    return best_pca, pca_df

# Training und Ausgabe der Prognosemodelle / Gebe die Statistiken und Ergebnisse der Prognosen durch
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird
# methode: "lr", "pr", "sv", "knn", "arima", "arimax" - Das zu nutzende Modell
# normalize: True, False - Ob Daten normalisiert werden
def compare_result2(daten, methode, normalize=True):
    # PCA mit niedrigstem MSE finden
    best_pca, pca_df = find_best_pca(daten, methode, normalize=True)

    # Prognose mit PCA
    x_train_pca = pca_df.drop(["co2"], axis=1)[:-4]
    x_test_pca = pca_df.drop(["co2"], axis=1)[-4:]
    y_train_pca = pca_df['co2'][:-4]
    y_test_pca = pca_df['co2'][-4:]

    # Modell mit PCA trainieren
    y_pred_pca = modell2(methode, x_train_pca, x_test_pca, y_train_pca, y_test_pca, daten)
    
    # MSE, RMSE, R², Bias und Varianz berechnen
    mae_pca = mean_absolute_error(y_test_pca, y_pred_pca)
    rmse_pca = np.sqrt(mean_squared_error(y_test_pca, y_pred_pca))
    r2_pca = r2_score(y_test_pca, y_pred_pca)
    var_pca = np.var(y_pred_pca)
    bias_pca = np.mean(y_pred_pca - y_test_pca)
    
    # Ausgabe der Metriken
    print("MAE mit PCA:", round(mae_pca, 2))
    print("RMSE mit PCA:", round(rmse_pca, 2))
    print("R² mit PCA:", round(r2_pca, 2))
    print("Varianz mit PCA:", round(var_pca, 2))
    print("Bias mit PCA:", round(bias_pca, 2))

    # Neue Daten für einen Vergleich der Daten vorbereiten
    neue_tabelle = pd.DataFrame({'PCA y': y_pred_pca, "Co2 Emissionen": y_test_pca})
    d = {'jahr': daten["jahr"][-4:], 'Co2 Wert': daten["co2"][-4:], "PCA y": daten["co2"][-4:]}
    df = pd.DataFrame(data=d)
    df["PCA y"][-4:] = neue_tabelle["PCA y"]

    # Naive Modelle & Mergen der Daten
    naiv=pd.DataFrame({"jahr": [2019, 2020, 2021, 2022],"Naiv 1": [754.408, 754.408, 754.408, 754.408], "Naiv 2": [795.557, 800.340, 785.616, 754.408]})
    df= pd.merge(df, naiv, on="jahr")
    return df

# Ausgabe der Einflüsse der einzelnen Variablen auf die PCA
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird
# methode: "lr", "pr", "sv", "knn", "arima", "arimax" - Das zu nutzende Modell
# n_components: 2 - 9 - Die Anzahl an Hauptkomponenten festlegen
# Contribution: 1, 2 - 1 steht für den Einfluss einer Variable auf alle Hauptkomponenten, 2 steht für den Einfluss auf einzelne Hauptkomponenten
# normalize: True, False - Ob Daten normalisiert werden
def relevant_variables_per(daten, methode, n_components, Contribution, normalize=True):
    pca_ausgaben=1
    # PCA - Dimensionen der Datensätze reduzieren
    x = daten.drop(["co2"], axis=1)

    if normalize:
        x = StandardScaler().fit_transform(x)

    x = pd.DataFrame(x, columns=daten.columns.drop("co2"))

    # Liste für MAE-Werte
    mae_values = []

    # PCA durchführen
    pca = PCA(n_components=n_components)
    PC = pca.fit_transform(x)
    principalDF = pd.DataFrame(data=PC, columns=[f'pc{i}' for i in range(1, n_components + 1)])
    pca_df = pd.concat([principalDF, daten[['co2']]], axis=1)

    # Festlegen des Trainings- und Testdatenbereichs
    x_train = pca_df.drop(["co2"], axis=1)[:-4]
    x_test = pca_df.drop(["co2"], axis=1)[-4:]
    y_train = pca_df['co2'][:-4]
    y_test = pca_df['co2'][-4:]

    # Ausgewählte Modellprognose
    y_pred = modell2(methode, x_train, x_test, y_train, y_test, daten)

    # Berechnung des Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    mae_values.append(mae)

    # Ausgabe des MAE-Werts
    print(f"Anzahl der PCA-Komponenten: {n_components}, MAE: {mae}")

    if Contribution == 1:
        # 1. Einfluss der Variablen über alle Hauptkomponenten
        # Variablen mit höchstem Beitrag zu den ausgewählten PCA-Komponenten finden
        # Mithilfe von Chatgpt generiert
        selected_components = [f'pc{i}' for i in range(1, n_components + 1)]
        variable_contributions = pd.DataFrame(
            data=pca.components_,
            columns=x.columns,
            index=selected_components
        ).abs()

        # Prozentsatz der Wichtigkeit der Variablen über alle ausgewählten Komponenten berechnen
        variable_importance_percentage = (variable_contributions / variable_contributions.sum(axis=1).to_numpy().reshape(-1, 1)) * 100

        # Gesamtprozentsatz der Wichtigkeit jeder Variable berechnen (über alle Komponenten)
        variable_total_importance_percentage = variable_importance_percentage.sum()/n_components

        # Sortieren der Variablen nach Gesamtprozentsatz der Wichtigkeit
        sorted_variables = variable_total_importance_percentage.sort_values(ascending=False)

        # Ausgabe der Variablen und ihrer prozentualen Wichtigkeit
        print("Wichtigste Variablen über alle ausgewählten PCA-Komponenten:")
        for variable, importance in sorted_variables.iteritems():
            print(f"Variable: {variable}, Wichtigkeit: {importance:.2f}%")

    elif Contribution == 2:
        # 2. Einfluss der Variablen über einzelne Hauptkomponenten
        # Variablen mit höchstem Beitrag zu den ausgewählten PCA-Komponenten finden
        # Mithilfe von Chatgpt generiert
        selected_components = [f'pc{i}' for i in range(1, n_components + 1)]
        highest_contributions = pd.DataFrame(
            data=pca.components_,
            columns=x.columns,
            index=selected_components
        )

        # Prozentsatz der Wichtigkeit der Variablen über alle ausgewählten Komponenten berechnen
        variable_importance_percentage = (highest_contributions.abs() / highest_contributions.abs().sum(axis=1).values[:, None]) * 100

        # Ausgabe der prozentualen Gewichte der Variablen für jede Hauptkomponente
        print(f"Prozentuale Gewichte der Variablen für {n_components} ausgewählte PCA-Komponenten:")
        for component in selected_components:
            print(f"PCA-Komponente {component}")
            print(variable_importance_percentage.loc[component].sort_values(ascending=False))
            print()
    else:
        pass

# PLotten der Modelle LR, PR, SVR, Arima und Arimax zum direkten Vergleich
# daten: daten1950, daten1973, daten1990, daten2006 - Welcher Datensatz genutzt wird
# Die Daten sind sehr redundant, da mir in anderer Form mit Schleifen nicht alle Lineplots korrekt angezeigt wurden
def plotten(daten):
    # Erzeugen von Werten in den Prognosejahren 2019-2022
    lr=compare_result2(daten, "lr", normalize=True)
    pr=compare_result2(daten, "pr", normalize=True)
    sv=compare_result2(daten, "sv", normalize=True)
    knn=compare_result2(daten, "knn", normalize=True)
    ama=compare_result2(daten, "arima", normalize=True)
    amax=compare_result2(daten, "arimax", normalize=True)
    
    # Umbenennung der Spalten in den Prognosespalten
    lr= lr.rename(columns={'PCA y': 'LR'})
    pr= pr.rename(columns={'PCA y': 'PR'})
    sv= sv.rename(columns={'PCA y': 'SVR'})
    knn= knn.rename(columns={'PCA y': 'KNN'})
    ama= ama.rename(columns={'PCA y': 'Arima'})
    amax= amax.rename(columns={'PCA y': 'Arimax'})
    
    # Mergen der Daten zu einem Dataframe
    df = pd.merge(lr, pr, on=["jahr", "Co2 Wert", "Naiv 1", "Naiv 2"]) \
    .merge(sv, on=["jahr", "Co2 Wert", "Naiv 1", "Naiv 2"]) \
    .merge(ama, on=["jahr", "Co2 Wert", "Naiv 1", "Naiv 2"]) \
    .merge(amax, on=["jahr", "Co2 Wert", "Naiv 1", "Naiv 2"]) \
    .merge(knn, on=["jahr", "Co2 Wert", "Naiv 1", "Naiv 2"])
    
    # Identische Datenströme erzeugen
    co=daten[["co2", "jahr"]][-8:-4]
    co = co.assign(LR=co["co2"])
    co = co.assign(PR=co["co2"])
    co = co.assign(SVR=co["co2"])
    co = co.assign(KNN=co["co2"])
    co = co.assign(Arima=co["co2"])
    co = co.assign(Arimax=co["co2"])
    co = co.assign(Naiv1=co["co2"])
    co = co.assign(Naiv2=co["co2"])
    co = co.rename(columns={'co2': 'Co2 Wert', "Naiv1": "Naiv 1", "Naiv2": "Naiv 2"})

    # Vermischen der Daten bis 2018
    df=pd.concat([co, df], ignore_index=True)
    
    # Plot vergrößern
    plt.figure(figsize=(14, 8))

    # Daten für den Plot
    sns.lineplot(data=df, x="jahr", y="Naiv 1", linewidth=1, label="Naiv 1")
    sns.lineplot(data=df, x="jahr", y="Naiv 2", linewidth=1, label="Naiv 2")
    sns.lineplot(data=df, x="jahr", y="Co2 Wert", linewidth=1, linestyle='dashed', label="Co2")
    sns.lineplot(data=df, x="jahr", y="LR", linewidth=1, label="LR")
    sns.lineplot(data=df, x="jahr", y="PR", linewidth=1, label="PR")
    sns.lineplot(data=df, x="jahr", y="KNN", linewidth=1, label="KNN")
    sns.lineplot(data=df, x="jahr", y="SVR", linewidth=1, label="SVR")
    sns.lineplot(data=df, x="jahr", y="Arima", linewidth=1, label="Arima")
    sns.lineplot(data=df, x="jahr", y="Arimax", linewidth=1, label="Arimax")

    # Plot erstellen
    plt.xlabel('Jahr')
    plt.ylabel('Co2 (in Mio. Tonnen)')
    plt.title('Vergleich der Prognosemodelle')
    plt.legend()

    # Anzeigen des Plots
    plt.show()